# Regularized regression: US county-level sociodemographic and health resource data (2018-2019)

For this project I will attempt to model the prevalence of morbidity at the county level as a function of socioeconomic, demographic and health care related features.

An initial round of manual feature selection will be used to discard clearly redundant, confounding or unnecessary features before EDA.

This dataset contains count data. For example: 100,000 people with a heart condition in Rhode Island vs Texas is two very different things. Texas has ~30x the population; keep this consideration in mind during the analysis.

## Notebook set-up

Handle imports of necessary modules up-front.

In [1]:
# Standard library imports
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge

# Set display options for pandas
pd.set_option('display.max_rows', 100)

## 1. Data loading

### 1.1. Load

In [2]:
data_url = 'https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv'
data_df = pd.read_csv(data_url, sep=',')

### 1.2. Save local copy

In [3]:
Path('../data/raw').mkdir(parents=True, exist_ok=True)
data_df.to_parquet('../data/raw/demographic_health_data.parquet', index=False)

### 1.3. Inspect

In [4]:
data_df.head().transpose()

,0,1,2,3,4
fips,1001,1003,1005,1007,1009
TOT_POP,55601,218022,24881,22400,57840
0-9,6787,24757,2732,2456,7095
0-9 y/o % of total pop,12.206615,11.355276,10.980266,10.964286,12.266598
19-Oct,7637,26913,2960,2596,7570
...,...,...,...,...,...
CKD_prevalence,3.1,3.2,4.5,3.3,3.4
CKD_Lower 95% CI,2.9,3.0,4.2,3.1,3.2
CKD_Upper 95% CI,3.3,3.5,4.8,3.6,3.7
CKD_number,1326,5479,887,595,1507


In [5]:
data_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 108 columns):
 #    Column                                                                         Non-Null Count  Dtype  
---   ------                                                                         --------------  -----  
 0    fips                                                                           3140 non-null   int64  
 1    TOT_POP                                                                        3140 non-null   int64  
 2    0-9                                                                            3140 non-null   int64  
 3    0-9 y/o % of total pop                                                         3140 non-null   float64
 4    19-Oct                                                                         3140 non-null   int64  
 5    10-19 y/o % of total pop                                                       3140 non-null   float64
 6    20-29         

## 2. Initial feature selection

In [6]:
# Save the label separately, so we don't lose or compromise it
label = data_df['anycondition_prevalence']
data_df.drop(columns=['anycondition_prevalence'], inplace=True)

### 2.1. Age features

In [ ]:
# Select age-related features
age_features = data_df.columns[2:20]
age_df = data_df[age_features].copy()

# Remove age count features, keeping only percentages
feature_drops = age_df.filter(regex = '^\d+-\d+$').columns
age_df.drop(columns=feature_drops, inplace=True)
age_df.drop(columns=['19-Oct', '80+'], inplace=True)

age_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   0-9 y/o % of total pop    3140 non-null   float64
 1   10-19 y/o % of total pop  3140 non-null   float64
 2   20-29 y/o % of total pop  3140 non-null   float64
 3   30-39 y/o % of total pop  3140 non-null   float64
 4   40-49 y/o % of total pop  3140 non-null   float64
 5   50-59 y/o % of total pop  3140 non-null   float64
 6   60-69 y/o % of total pop  3140 non-null   float64
 7   70-79 y/o % of total pop  3140 non-null   float64
 8   80+ y/o % of total pop    3140 non-null   float64
dtypes: float64(9)
memory usage: 220.9 KB


### 2.2. Ethnicity features

In [17]:
# Select Ethnicity-related features
ethnicity_features = data_df.columns[20:32]
ethnicity_df = data_df[ethnicity_features].copy()

# Remove count features, keeping only percentages
features = ethnicity_df.filter(regex = '^\%').columns
ethnicity_df = ethnicity_df[features]

ethnicity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   % White-alone        3140 non-null   float64
 1   % Black-alone        3140 non-null   float64
 2   % NA/AI-alone        3140 non-null   float64
 3   % Asian-alone        3140 non-null   float64
 4   % Hawaiian/PI-alone  3140 non-null   float64
 5   % Two or more races  3140 non-null   float64
dtypes: float64(6)
memory usage: 147.3 KB


### 2.3. Population features

In [ ]:
population_features = ['TOT_POP', 'POP_ESTIMATE_2018', 'N_POP_CHG_2018', 'R_birth_2018', 'R_death_2018']
population_df = data_df[population_features].copy()
population_df.head()

In [ ]:
# # Do the test-train split
# training_df, testing_df=train_test_split(
#     data_df,
#     test_size=0.25,
#     random_state=315
# )

## 2. EDA

### 2.1. Baseline model performance

In [ ]:
anycondition_rates=training_df['anycondition_number']/training_df['TOT_POP']
mean_anycondition_rate=np.mean(anycondition_rates)
print(f'Mean any condition rate: {mean_anycondition_rate:.2f}')

rmse=root_mean_squared_error(anycondition_rates, [mean_anycondition_rate]*len(training_df))
print(f'Mean any condition rate model RMSE: {rmse:.2f}')

### 2.2. Inital feature selection

#### 2.2.1. Drop pathology related features

In [ ]:
training_labels=training_df['anycondition_number']/training_df['TOT_POP']
testing_labels=testing_df['anycondition_number']/testing_df['TOT_POP']

path_features=training_df.columns[:83]
training_df=training_df[path_features]
testing_df=testing_df[path_features]

#### 2.2.2. Initial feature selection

In [ ]:
string_features=['COUNTY_NAME', 'STATE_NAME']
ordinal_encoder=OrdinalEncoder()
training_df[string_features]=ordinal_encoder.fit_transform(training_df[string_features])

linear_model=LinearRegression()
selector=RFE(linear_model, n_features_to_select=21, step=1)
selector=selector.fit(training_df, training_labels)

training_features=training_df.loc[:, selector.get_support()].copy()
testing_features=testing_df.loc[:, selector.get_support()].copy()

training_features.head().transpose()

### 2.3. Feature distributions and cleaning

In [ ]:
new_columns={
    'Active Patient Care Primary Care Physicians per 100000 Population 2018 (AAMC)': 'Primary physicians',
    'Active Patient Care General Surgeons per 100000 Population 2018 (AAMC)': 'General surgons'
}

training_features.rename(columns=new_columns, inplace=True)
testing_features.rename(columns=new_columns, inplace=True)

In [ ]:
plt.title('Label distribution')
plt.hist(training_labels, color='black', bins=30)
plt.xlabel('Incidence rate of any condition')
plt.ylabel('Count')
plt.show()

In [ ]:
features=list(training_features.columns)

rows=len(features) // 3

fig, axs=plt.subplots(rows,3, figsize=(8,2.5*rows))
axs=axs.flatten()

for i, feature in enumerate(features):
    axs[i].set_title(feature)
    axs[i].hist(training_features[feature], color='black', bins=30)
    axs[i].set_ylabel('Count')

fig.tight_layout()
fig.show()

### 2.4. Feature interactions & selection

In [ ]:
features=list(training_features.columns)

rows=len(features) // 3

fig, axs=plt.subplots(rows,3, figsize=(8,2.5*rows))
axs=axs.flatten()

for i, feature in enumerate(features):
    axs[i].set_title(feature)
    axs[i].scatter(training_features[feature], training_labels, s=0.2, color='black')
    axs[i].set_ylabel('Count')

fig.tight_layout()
fig.show()

### 2.5. Feature encoding & scaling

In [ ]:
scaler=StandardScaler()
scaler.fit(training_features)
scaled_training_features=scaler.transform(training_features)
scaled_testing_features=scaler.transform(testing_features)

training_features=pd.DataFrame(scaled_training_features, columns=training_features.columns)
testing_features=pd.DataFrame(scaled_testing_features, columns=testing_features.columns)

training_features.describe().transpose()

## 3. Linear model training

In [ ]:
linear_model=LinearRegression()
result=linear_model.fit(training_features, training_labels)

train_predictions=linear_model.predict(training_features)
train_rmse=root_mean_squared_error(training_labels, train_predictions)

test_predictions=linear_model.predict(testing_features)
test_rmse=root_mean_squared_error(testing_labels, test_predictions)

print(f'Prediction RMSE: training: {train_rmse:.4f}, testing: {test_rmse:.4f}')

In [ ]:
fig, axs=plt.subplots(1,2, figsize=(8,4), sharex=True, sharey=True)
axs=axs.flatten()

fig.suptitle('Split model evaluation')

axs[0].set_title('Training data: true vs predicted\nany condition rate')
axs[0].scatter(training_labels, train_predictions, color='black', s=5)
axs[0].set_xlabel('True rate')
axs[0].set_ylabel('Predicted rate')

axs[1].set_title('Testing data: true vs predicted\nany condition rate')
axs[1].scatter(testing_labels, test_predictions, color='black', s=5)
axs[1].set_xlabel('True rate')
axs[1].set_ylabel('Predicted rate')

fig.tight_layout()
fig.show()

## 4. Model regularization

In [ ]:
%%time

penalties=[0.0000125, 0.000025, 0.00005, 0.0001, 0.0002, 0.0004, 0.0008, 0.0016, 0.0032, 0.0064, 0.0128, 0.0256, 0.1024, 0.2048]
train_rmse_values=[]
test_rmse_values=[]

for penalty in penalties:
    lasso_model=Lasso(alpha=penalty, max_iter=5000)
    result=lasso_model.fit(training_features, training_labels)

    train_predictions=lasso_model.predict(training_features)
    train_rmse=root_mean_squared_error(training_labels, train_predictions)
    train_rmse_values.append(train_rmse)

    test_predictions=lasso_model.predict(testing_features)
    test_rmse=root_mean_squared_error(testing_labels, test_predictions)
    test_rmse_values.append(test_rmse)

    print(f'Prediction RMSE: training: {train_rmse:.4f}, testing: {test_rmse:.4f}')

print()

In [ ]:
plt.title('Regression RMSE vs L1 penalty')
plt.plot(penalties, train_rmse_values, color='black', label='Training')
plt.plot(penalties, test_rmse_values, color='red', label='Testing')
plt.xlabel('L1 penalty')
plt.xscale('log', base=2)
plt.ylabel('RMSE')
plt.legend(loc='best')
plt.show()

## 5. Hyperparameter optimization

In [ ]:
%%time

penalties=[1, 10, 100, 1000, 10000]
train_rmse_values=[]
test_rmse_values=[]

for penalty in penalties:
    ridge_model=Ridge(alpha=penalty, max_iter=5000)
    result=ridge_model.fit(training_features, training_labels)

    train_predictions=ridge_model.predict(training_features)
    train_rmse=root_mean_squared_error(training_labels, train_predictions)
    train_rmse_values.append(train_rmse)

    test_predictions=ridge_model.predict(testing_features)
    test_rmse=root_mean_squared_error(testing_labels, test_predictions)
    test_rmse_values.append(test_rmse)

    print(f'Prediction RMSE: training: {train_rmse:.4f}, testing: {test_rmse:.4f}')

print()

In [ ]:
plt.title('Regression RMSE vs L1 penalty')
plt.plot(penalties, train_rmse_values, color='black', label='Training')
plt.plot(penalties, test_rmse_values, color='red', label='Testing')
plt.xlabel('L1 penalty')
plt.xscale('log', base=2)
plt.ylabel('RMSE')
plt.legend(loc='best')
plt.show()

## 6. Final model evaluation

In [ ]:
lasso_model=Lasso(alpha=2**-9.7, max_iter=5000)
result=lasso_model.fit(training_features, training_labels)

train_predictions=lasso_model.predict(training_features)
train_rmse=root_mean_squared_error(training_labels, train_predictions)

test_predictions=lasso_model.predict(testing_features)
test_rmse=root_mean_squared_error(testing_labels, test_predictions)

print(f'Prediction RMSE: training: {train_rmse:.4f}, testing: {test_rmse:.4f}')